In [1]:
# import os
# #import magic
# import urllib.request
# from app import app
# from flask import Flask, flash, request, redirect, render_template
# from werkzeug.utils import secure_filename
# import sys
# # app = Flask(__name__)
# ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif','mp4'])

# def allowed_file(filename):
# 	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
	
# @app.route('/')
# def upload_form():
# 	return render_template('upload.html')

# # run_flag = False

# # @app.route('/obstacle')
# # def obstacle():
# # 	path = "/home/akhilesh/Server/frame15.jpg"
# # 	cap = cv2.imread(path)
# # 	cv2.imshow("frame",cap)
# # 	cv2.waitKey(0)

# @app.route('/', methods=['POST'])
# def upload_file():
# 	if request.method == 'POST':
#         # check if the post request has the file part
# 		if 'file' not in request.files:
# 			flash('No file part')
# 			return redirect(request.url)
# 		file = request.files['file']
# 		if file.filename == '':
# 			flash('No file selected for uploading')
# 			return redirect(request.url)
# 		if file and allowed_file(file.filename):
# 			filename = secure_filename(file.filename)
# 			file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
# 			flash('File successfully uploaded')
# 			return redirect('/')
# 		else:
# 			flash('Allowed file types are txt, pdf, png, jpg, jpeg, gif,mp4')
# 			return redirect(request.url)

	

# # @app.route('/obstacle')
# # def forward():
# # 	global run_flag
# # 	print("Forward!")
# # 	run_flag = True
# # 	while run_flag :
		
# 		# return 'Alexabot moved forward!'
#     #place code here from Obstacle_avoidRobo.py
  	








# # @app.route('/')
# # def index():
# # return 'Hello world'


# # @app.route('/stop')
# # def stop():
# # 	global run_flag
# # 	run_flag = False



# # @app.route('/quit')
# # def quit():
# # 	func = request.environ.get('werkzeug.server.shutdown')
# # 	func()
# # 	return "Quitting..."

# # if __name__ == '__main__':


# def shutdown_server():
#     func = request.environ.get('werkzeug.server.shutdown')
#     if func is None:
#         raise RuntimeError('Not running with the Werkzeug Server')
#     else:
#     	shutdown()

# @app.route('/shutdown', methods=['POST'])
# def shutdown():
#     shutdown_server()
#     return 'Server shutting down...'

# if __name__ == "__main__":
#     # app.run()
# 	app.run()

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
print(tf.__version__)

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
try:
    import cv2
except:
    import sys
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
    import cv2
import recognize
from gtts import gTTS
from playsound import playsound


language = 'en'

image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

f = open('train_captions_1.pickle', 'rb') #read pickled captions from training, download from drive if trained on colab
train_captions = pickle.load(f)
# Choose the top 5000 words from the vocabulary
top_k = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,oov_token="<unk>",filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = len(tokenizer.word_index) + 1
# num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64
max_length = 49

class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 64, hidden_size)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 64, 1)
    # you get 1 at the last axis because you are applying score to self.V
        attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights
    
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x
    
class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
    # defining attention as a separate model
        context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
        output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
        x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
        x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
        x = self.fc2(x)

        return x, state, attention_weights
    
    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))
    
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

checkpoint_path = "checkpoint_2.1"   #ADD CHECKPOINT FILE DOWNLOADED FROM DRIVE AFTER TRAINING ON COLAB.
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

ckpt.restore(ckpt_manager.latest_checkpoint)

def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    # temp_input = tf.expand_dims(load_image(image)[0], 0)
    # img_tensor_val = image_features_extract_model(image)
    # img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(image)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

#         attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

def preprocess(frame):
    img = np.expand_dims(frame, axis=0)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    feature = image_features_extract_model(img)
    feature = tf.reshape(feature, (feature.shape[0], -1, feature.shape[3]))
    return feature

2.0.0
[INFO] loading encodings...
[INFO] starting video stream...


In [3]:
# # import numpy as np
# # import cv2
# # import upload
# cap = cv2.VideoCapture(0)
# count = 0
# i = 0
# # import glob
# while(cap.isOpened()):
#     # Capture frame-by-frame
#     ret, frame = cap.read()
#     if ret == False:
#         break

#     # Our operations on the frame come here
#     # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     count +=1
#     # Display the resulting frame
#     while(count==150):
#         cv2.imwrite('test'+str(i)+'.jpg', frame)
#         print(count)
#         count=0
#         i += 1
#         # cv2.imshow("frame",frame)
#         # count += 1
#     int m = i
#     cv2.imshow("frame",frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# # for name in glob.glob('/home/akhilesh/Server/frame?.jpg'):
#     # print(name)
# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()


In [4]:
# import numpy as np
# import cv2
# import upload

cap = cv2.VideoCapture('hello1.mp4')
out = cv2.VideoWriter('output.avi', -1, 20.0, (640,480))
count = 0
i = 0
# import glob
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == False:
        break
    out.write(frame)
    # Our operations on the frame come here
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    count +=1
    # Display the resulting frame
    while(count==150):
        cv2.imwrite('test'+str(i)+'.jpg', frame)
        print(count)
        count=0
        i += 1
        # cv2.imshow("frame",frame)
        # count += 1
    m = i
    
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# for name in glob.glob('/home/akhilesh/Server/frame?.jpg'):
    # print(name)
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
i=0
for i in range(m):
    file = 'test{}'.format(i)
    image = cv2.imread('/home/ameya/rakathon/'+file+'.jpg')
#     image = cv2.imread('/home/avitra/rakathon/test_imgs/frame15.jpg')
    name = recognize.recognize(image)
    feature = preprocess(image)
    result, _ = evaluate(feature)
#     print("The person is",name)
    text = ''
    p = ''
    for i in result:
        text += i+' '
    for j in name:
        p=p+' '+j
    if len(name)>0:
        text = text[:text.find('<end>')]+' The person is '+p
        print(text)
    else:
        text = text[:text.find('<end>')]
        print(text)
#     print(text,"The person is",name)
 
    cv2.imshow('frame', cv2.resize(image, (int(image.shape[1]*0.2), int(image.shape[0]*0.2)), interpolation = cv2.INTER_AREA))
    cv2.waitKey(5000)
    cv2.destroyAllWindows()
    myobj = gTTS(text=text, lang=language, slow=False)
    myobj.save(file+'.mp3')
    #playsound(file+'.mp3')

150
150
150
150
150
150
150
a bouquet of leafy vegetables are in a vase with flowers 
a vase filled with roses inside a vase on a table 
a vase with a bunch of flowers 
a vase with a large vase with a large vase with a large vase with a large vase with a large vase with a large vase with a large vase with a large vase with a large vase with a large vase with a large vase with a large
a table with a table and chairs 
a table with a single table and a bar 
a table with a table with a bar 


In [5]:
# cap = cv2.VideoCapture(0)
# language = 'en'

In [6]:
# image_model = tf.keras.applications.InceptionV3(include_top=False,
#                                                 weights='imagenet')
# new_input = image_model.input
# hidden_layer = image_model.layers[-1].output

# image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [7]:
# f = open('train_captions_1.pickle', 'rb') #read pickled captions from training, download from drive if trained on colab
# train_captions = pickle.load(f)
# # Choose the top 5000 words from the vocabulary
# top_k = 10000
# tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,oov_token="<unk>",filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
# tokenizer.fit_on_texts(train_captions)
# train_seqs = tokenizer.texts_to_sequences(train_captions)
# tokenizer.word_index['<pad>'] = 0
# tokenizer.index_word[0] = '<pad>'
# # Create the tokenized vectors
# train_seqs = tokenizer.texts_to_sequences(train_captions)

In [8]:
# # Feel free to change these parameters according to your system's configuration

# BATCH_SIZE = 64
# BUFFER_SIZE = 1000
# embedding_dim = 256
# units = 512
# vocab_size = len(tokenizer.word_index) + 1
# # num_steps = len(img_name_train) // BATCH_SIZE
# # Shape of the vector extracted from InceptionV3 is (64, 2048)
# # These two variables represent that vector shape
# features_shape = 2048
# attention_features_shape = 64
# max_length = 49

In [9]:
# class BahdanauAttention(tf.keras.Model):
#     def __init__(self, units):
#         super(BahdanauAttention, self).__init__()
#         self.W1 = tf.keras.layers.Dense(units)
#         self.W2 = tf.keras.layers.Dense(units)
#         self.V = tf.keras.layers.Dense(1)

#     def call(self, features, hidden):
#     # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

#     # hidden shape == (batch_size, hidden_size)
#     # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
#         hidden_with_time_axis = tf.expand_dims(hidden, 1)

#     # score shape == (batch_size, 64, hidden_size)
#         score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

#     # attention_weights shape == (batch_size, 64, 1)
#     # you get 1 at the last axis because you are applying score to self.V
#         attention_weights = tf.nn.softmax(self.V(score), axis=1)

#     # context_vector shape after sum == (batch_size, hidden_size)
#         context_vector = attention_weights * features
#         context_vector = tf.reduce_sum(context_vector, axis=1)

#         return context_vector, attention_weights

In [10]:
# class CNN_Encoder(tf.keras.Model):
#     # Since you have already extracted the features and dumped it using pickle
#     # This encoder passes those features through a Fully connected layer
#     def __init__(self, embedding_dim):
#         super(CNN_Encoder, self).__init__()
#         # shape after fc == (batch_size, 64, embedding_dim)
#         self.fc = tf.keras.layers.Dense(embedding_dim)

#     def call(self, x):
#         x = self.fc(x)
#         x = tf.nn.relu(x)
#         return x

In [11]:
# class RNN_Decoder(tf.keras.Model):
#     def __init__(self, embedding_dim, units, vocab_size):
#         super(RNN_Decoder, self).__init__()
#         self.units = units

#         self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
#         self.gru = tf.keras.layers.GRU(self.units,
#                                    return_sequences=True,
#                                    return_state=True,
#                                    recurrent_initializer='glorot_uniform')
#         self.fc1 = tf.keras.layers.Dense(self.units)
#         self.fc2 = tf.keras.layers.Dense(vocab_size)

#         self.attention = BahdanauAttention(self.units)

#     def call(self, x, features, hidden):
#     # defining attention as a separate model
#         context_vector, attention_weights = self.attention(features, hidden)

#     # x shape after passing through embedding == (batch_size, 1, embedding_dim)
#         x = self.embedding(x)

#     # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
#         x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

#     # passing the concatenated vector to the GRU
#         output, state = self.gru(x)

#     # shape == (batch_size, max_length, hidden_size)
#         x = self.fc1(output)

#     # x shape == (batch_size * max_length, hidden_size)
#         x = tf.reshape(x, (-1, x.shape[2]))

#     # output shape == (batch_size * max_length, vocab)
#         x = self.fc2(x)

#         return x, state, attention_weights
    
#     def reset_state(self, batch_size):
#         return tf.zeros((batch_size, self.units))

In [12]:
# encoder = CNN_Encoder(embedding_dim)
# decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [13]:
# optimizer = tf.keras.optimizers.Adam()
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=True, reduction='none')

# def loss_function(real, pred):
#     mask = tf.math.logical_not(tf.math.equal(real, 0))
#     loss_ = loss_object(real, pred)

#     mask = tf.cast(mask, dtype=loss_.dtype)
#     loss_ *= mask

#     return tf.reduce_mean(loss_)

In [14]:
# checkpoint_path = "checkpoint_2.1"   #ADD CHECKPOINT FILE DOWNLOADED FROM DRIVE AFTER TRAINING ON COLAB.
# ckpt = tf.train.Checkpoint(encoder=encoder,
#                            decoder=decoder,
#                            optimizer = optimizer)
# ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [15]:
# ckpt.restore(ckpt_manager.latest_checkpoint)

In [16]:
# def evaluate(image):
#     attention_plot = np.zeros((max_length, attention_features_shape))

#     hidden = decoder.reset_state(batch_size=1)

#     # temp_input = tf.expand_dims(load_image(image)[0], 0)
#     # img_tensor_val = image_features_extract_model(image)
#     # img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

#     features = encoder(image)

#     dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
#     result = []

#     for i in range(max_length):
#         predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

# #         attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

#         predicted_id = tf.argmax(predictions[0]).numpy()
#         result.append(tokenizer.index_word[predicted_id])

#         if tokenizer.index_word[predicted_id] == '<end>':
#             return result, attention_plot

#         dec_input = tf.expand_dims([predicted_id], 0)

#     attention_plot = attention_plot[:len(result), :]
#     return result, attention_plot

In [17]:
# def preprocess(frame):
#     img = np.expand_dims(frame, axis=0)
#     img = tf.image.resize(img, (299, 299))
#     img = tf.keras.applications.inception_v3.preprocess_input(img)
#     feature = image_features_extract_model(img)
#     feature = tf.reshape(feature, (feature.shape[0], -1, feature.shape[3]))
#     return feature

In [18]:
# image = cv2.imread('/home/avitra/rakathon/uploaded/testimg1.jpg')
# # cv2.imshow('frame', image)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()
# name = recognize.recognize(image)
# feature = preprocess(image)
# result, _ = evaluate(feature)



In [19]:
# cap = cv2.VideoCapture(2)
# frame_count = 0
# # placeholders = [' a man', ' a person', ' a boy']
# while True:
#     ret, frame = cap.read()
#     if cv2.waitKey(1)==27:
#         break
#     if not ret:
#         pass
#     else:
#         frame_count+=1
#         cv2.imshow('frame', frame)
#         if frame_count%250==0:
#             name = recognize.recognize(frame)
#             feature = preprocess(frame)
#             result, _ = evaluate(feature)
#             text = ''
#             for i in result:
#                 text += i+' '
#             text = text[:text.find('<end>')]
#             print(text)
# #             if len(name)>0:
# #                 for i in placeholders:
# #                     if i not in text:
# #                         pass
# #                     else:
# #                         name_index = text.find(i)+1
# #                         text = text[:name_index]+'{}'.format(name[0])+text[name_index+len(i)-1:]
# #             print(text)
#             myobj = gTTS(text=text, lang=language, slow=False)
#             myobj.save("audio.mp3")
#             playsound('audio.mp3')
# cap.release()
# cv2.destroyAllWindows()